## Things to do:
1. Country to Country: sankey diagram with (a) sector in the middle (b) umbrella in the middle
2. *Liars*:  disbursed - committed (color shows # of failures): square tree 
3. *Corrupts*: total project cost- disbursed (color shows # of failures): square tree [do it by filtering for projects that were completed, i.e. start_date and end_date not zero]
4. **Association rule mining** on categorical variables associated with Nepal. (Present as word cloud with importance based on lift)
5. Pick some variables and predict whether a project is completed or not. rank the variables by importance. 

In [224]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
df=pd.read_csv('/Users/sumangiri/Downloads/aiddata2-1_research_release.csv')

In [225]:
df=df.fillna(0)
df.head(2)

,aiddata_id,year,commitment_date,start_date,end_date,donor,donor_iso2,donor_iso3,umbrella,donor_project_id,...,investment_project,aiddata_purpose_code,aiddata_purpose_name,aiddata_activity_code,aiddata_activity_name,aiddata_feasibility_study,aiddata_technical_assistance,coalesced_purpose_code,coalesced_purpose_name,aiddata_is_conflict
0,21235774,2004,0,0,0,Belgium,BE,BEL,0,1840171,...,0,0,0,0,0,0,0,92010,Support to national ngos,0
1,94726192,2010,0,2010-06-07 00:00:00,2010-12-07 00:00:00,United Kingdom,GB,GBR,0,201848-101,...,0,0,0,0,0,0,0,72010,Emergency/distress relief,0


In [226]:
df=df[df.aiddata_id!=2445044]

In [254]:
df1=df[['donor',\
'recipient',\
'sector',\
'disbursement_amount_usd_nominal',\
'commitment_amount_usd_nominal',\
'received_amount_usd_nominal',\
'total_project_cost_usd_nominal']].groupby(['donor','recipient']).sum().reset_index()\
.sort_values('disbursement_amount_usd_nominal',ascending=[0])

In [255]:
import numpy as np
df1['com_dis']=np.abs(1-np.round(df1.disbursement_amount_usd_nominal*1.0 / (1+df1.commitment_amount_usd_nominal),2)) #liars
df1['dis_pc']=np.abs(1-np.round(df1.total_project_cost_usd_nominal*1.0/(1+df1.disbursement_amount_usd_nominal),2))   #corrupt 
df1=df1.fillna(0)

## Countries that commit to more than what they disburse: liars

In [336]:
df1['com_dis'][(df1.com_dis>1e100)]=0
df2=df1[(df1.com_dis>0)&(df1.com_dis<100.01)&(df1.disbursement_amount_usd_nominal>1e4)]

In [337]:
#get donors who have at least 5 projects that meet our criteria
filter_list_dnr=df2[['donor','com_dis']].groupby('donor').count().sort_values(by='com_dis', ascending=[0])\
.reset_index()
filter_list_dnr=filter_list_dnr['donor'].loc[filter_list_dnr.com_dis>4]

In [345]:
dnr_list= df2[['donor','com_dis']].loc[df2.donor.isin(filter_list_dnr)]\
.groupby('donor').mean().sort_values(by='com_dis', ascending=[0])\
.reset_index().donor[0:10]

In [364]:
df2[['donor','com_dis']].loc[df2.donor.isin(filter_list_dnr)]\
.groupby('donor').mean().sort_values(by='com_dis', ascending=[0])\
.reset_index()


,donor,com_dis
0,Andean Development Corporation (CAF),0.998000
1,European Communities (EC),0.945500
2,OPEC Fund for International Development (OFID),0.912432
3,Arab Bank for Economic Development in Africa (...,0.856286
4,Australia,0.809159
5,Switzerland,0.802164
6,Canada,0.785597
7,Denmark,0.784717
8,World Bank - International Bank for Reconstruc...,0.761558
9,Italy,0.750797


In [348]:
com_dis_df=df2[df2.donor.isin(dnr_list)]

In [351]:
mapping_com_dis={'World Bank - International Bank for Reconstruction and Development (IBRD)':'World Bank (IBRD)',\
        'Andean Development Corporation (CAF)': 'Andean Dev Corp (CAF)',\
        'OPEC Fund for International Development (OFID)':'OPEC (OFID)',\
        'Arab Bank for Economic Development in Africa (BADEA)':'Arab Bank (BADEA)'}

In [352]:
com_dis_df['donor'] = com_dis_df['donor'].replace(mapping_com_dis)

In [354]:
com_dis_df.to_csv('treemap_com_dis.csv')

### Countries whose project cost does not add up to the amount disbursed for it: corrupts

In [304]:
#dont count instances where disb amt was 0 or cases with no project cost
df1['dis_pc'][(df1.dis_pc>1e100)]=0
df3=df1[(df1.dis_pc<100.01)&(df1.total_project_cost_usd_nominal>1e4)]

In [260]:
# df3[['donor','recipient','disbursement_amount_usd_nominal','dis_pc', 'commitment_amount_usd_nominal']]

In [305]:
#get recipients who have at least 4 projects that meet our criteria
filter_list=df3[['recipient','dis_pc']].groupby('recipient').count().sort_values(by='dis_pc', ascending=[0])\
.reset_index()
filter_list=filter_list['recipient'].loc[filter_list.dis_pc>2]

In [306]:
#get recipients who meet the 4 project criteria AND 
#have the largest average difference between project cost and disbursed amount

rcp_list= df3[['recipient','dis_pc']].loc[df3.recipient.isin(filter_list)]\
.groupby('recipient').mean()\
.sort_values(by='dis_pc', ascending=[0])\
.reset_index().recipient[0:10]

In [307]:
dis_pc_df=df3[df3.recipient.isin(rcp_list)]

In [308]:
dis_pc_df.donor.unique()

array(['World Bank - International Development Association (IDA)',
       'Arab Fund for Economic & Social Development (AFESD)', 'Kuwait',
       'World Bank - International Bank for Reconstruction and Development (IBRD)',
       'OPEC Fund for International Development (OFID)',
       'Arab Bank for Economic Development in Africa (BADEA)',
       'United Arab Emirates', 'Nordic Development Fund (NDF)',
       'World Bank - Managed Trust Funds'], dtype=object)

In [309]:
mapping={'World Bank - International Bank for Reconstruction and Development (IBRD)':'World Bank (IBRD)', \
         'World Bank - International Development Association (IDA)':'World Bank (IDA)',\
         'Arab Fund for Economic & Social Development (AFESD)':'Arab Fund (AFESD)',\
         'Arab Bank for Economic Development in Africa (BADEA)': 'Arab Bank (BADEA)',\
         'OPEC Fund for International Development (OFID)': 'OPEC Fund (OFID)',\
         'Nordic Development Fund (NDF)':'Nordi Development Fund (NDF)',\
         'Kuwait':'Kuwait'}
    

In [310]:
dis_pc_df.donor=dis_pc_df.donor.map(mapping)

In [311]:
dis_pc_df.fillna('Unknown',inplace='True')

In [312]:
dis_pc_df.to_csv('treemap_dis_pc.csv')